In [1]:
import requests
import geojson
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import shape
from pathlib import Path

/Users/ajalsingh/git/race-line-optimser/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### Scrape track data

In [2]:
# Directory to save track data
OUTPUT_DIR = Path("../track_data")
OUTPUT_DIR.mkdir(exist_ok=True)

In [3]:
# Define Overpass API endpoint
OVERPASS_URL = "https://overpass-api.de/api/interpreter"

# List of F1 tracks to query
TRACKS = [
    {"name": "Circuit de Monaco", "type": "relation"},
    {"name": "Silverstone Circuit", "type": "relation"},
    {"name": "Spa-Francorchamps", "type": "relation"},
    {"name": "Autodromo Nazionale di Monza", "type": "relation"},
    {"name": "Circuit of the Americas", "type": "relation"},
]

In [10]:
def query_overpass(track_name, track_type):
    """
    Query the Overpass API for a specific track.
    """
    query = f"""
    [out:json];
    {track_type}["name"="{track_name}"];
    out body;
    >;
    out skel qt;
    """
    response = requests.get(OVERPASS_URL, params={"data": query})
    response.raise_for_status()
    return response.json()


def save_geojson(track_name, data):
    """
    Save the track data to a GeoJSON file.
    """
    filename = OUTPUT_DIR / f"{track_name.replace(' ', '_')}.geojson"
    with open(filename, "w") as file:
        geojson.dump(data, file, indent=2)
    print(f"Saved GeoJSON for {track_name} to {filename}")


# Fetch data for all F1 tracks and save them as GeoJSON.

for track in TRACKS:
    print(f"Fetching data for {track['name']}...")
    try:
        data = query_overpass(track["name"], track["type"])
        save_geojson(track["name"], data)
    except Exception as e:
        print(f"Failed to fetch data for {track['name']}: {e}")

Fetching data for Circuit de Monaco...


Saved GeoJSON for Circuit de Monaco to ../track_data/Circuit_de_Monaco.geojson
Fetching data for Silverstone Circuit...
Saved GeoJSON for Silverstone Circuit to ../track_data/Silverstone_Circuit.geojson
Fetching data for Spa-Francorchamps...
Saved GeoJSON for Spa-Francorchamps to ../track_data/Spa-Francorchamps.geojson
Fetching data for Autodromo Nazionale di Monza...
Saved GeoJSON for Autodromo Nazionale di Monza to ../track_data/Autodromo_Nazionale_di_Monza.geojson
Fetching data for Circuit of the Americas...
Saved GeoJSON for Circuit of the Americas to ../track_data/Circuit_of_the_Americas.geojson


### Visualise tracks

In [4]:
# Directory containing the GeoJSON files
TRACK_DIR = Path("../track_data")


def load_geojson(file_path):
    """
    Load a GeoJSON file and extract its geometry.
    """
    with open(file_path, "r") as file:
        data = geojson.load(file)
    geometries = []
    for feature in data.get("features", []):
        geom = shape(feature["geometry"])
        geometries.append(geom)
    return geometries

In [7]:
files = list(TRACK_DIR.glob("*.geojson"))
num_files = len(files)
cols = 2
rows = (num_files + 1) // cols  # Calculate rows needed for the grid

for idx, file in enumerate(files):
    print(f"Visualizing {file.stem}...")
    geometries = load_geojson(file)
    for geom in geometries:
        if geom.geom_type == "LineString":
            x, y = geom.xy
            plt.plot(x, y, label=file.stem)
        elif geom.geom_type == "MultiLineString":
            for line in geom.geoms:
                x, y = line.xy
                plt.plot(x, y, label=file.stem)

    # plt.set_title(file.stem)
    # plt.set_xlabel("Longitude")
    # plt.set_ylabel("Latitude")
    # plt.legend(loc="upper right", fontsize="small")
    # plt.grid(True)

plt.tight_layout()
plt.show()

Visualizing Autodromo_Nazionale_di_Monza...
Visualizing Circuit_of_the_Americas...
Visualizing Spa-Francorchamps...
Visualizing Circuit_de_Monaco...
Visualizing Silverstone_Circuit...


<Figure size 640x480 with 0 Axes>